In [24]:
import numpy as np
from math import floor, ceil

# Part A

In [3]:
def query(votes, guess):
    return np.sum(np.logical_xor(votes, guess))

In [4]:
members = 20

for i in range(1000):
    
    votes = np.random.randint(low=0, high=2, size=members)
    partition = np.random.randint(low=0, high=members + 1)

    actual_no_votes_partition = partition - np.sum(votes[:partition])
    
    ones = np.ones((members,))
    first = query(votes, ones)

    ones[:partition] = 0
    second = query(votes, ones)

    guess_no_votes_partition = (partition - (second - first)) // 2
    
    assert guess_no_votes_partition == actual_no_votes_partition

# Part B

In [5]:
def find_no_voter(votes, i, j):
    
    num_no_voters = query(votes[i:j], np.ones(j - i))
    
    # If there are no no-voters in this subsequence
    if num_no_voters == 0:
        return None
    
    # If the subsequence only has a single person...
    if (j - i) == 1:
        
        # ... and that person votes no
        if num_no_voters == 1:
            return i
    
        # ... and that person votes yes
        if num_no_voters == 0:
            return None
    
    m = (i + j) // 2
    
    left_num_no_voters = query(votes[i:m], np.ones(m - i))
    right_num_no_voters = query(votes[m:j], np.ones(j - m))
    
    if left_num_no_voters > right_num_no_voters:
        return find_no_voter(votes, i, m)
    else:
        return find_no_voter(votes, m, j)    

In [6]:
members = 8

for i in range(1000):
        
    votes = np.random.randint(low=0, high=2, size=members)
    
    no_voter = find_no_voter(votes, 0, len(votes))
    
    if no_voter is not None:
        assert votes[no_voter] == 0
    else:
        assert members - np.sum(votes) == 0

# Part C

In [119]:
def group(A, n=5):
        
    for i in range(0, len(A), n):
        yield A[i:i+n]

In [91]:
def simple_median(A):
    
    n = len(A) 
    A_sorted = np.sort(A)
    
    return A_sorted[(n - 1) // 2]

In [97]:
def partition(A, p, r):
    
    x = A[r]
    i = p - 1
    
    for j in range(p, r):
        
        if A[j] <= x:
            i = i + 1
            A[i], A[j] = A[j], A[i]
    
    A[i + 1], A[r] = A[r], A[i + 1]

    return i + 1

In [98]:
def randomized_partition(A, p, r):
    
    i = np.random.randint(p, r)
    A[i], A[r] = A[r], A[i]
    
    return partition(A, p, r)

In [99]:
def modified_partition(A, p, r, i):
    
    A[i], A[r] = A[r], A[i]
    
    return partition(A, p, r)

In [116]:
def index_of(A, x):
    
    for i in range(len(A)):
        
        if A[i] == x:
            return i
    
    return -1

In [101]:
def randomized_select(A, p, r, i):
    
    if p == r:
        return A[p]
    
    q = randomized_partition(A, p, r)
    k = q - p + 1
    
    if i == k:
        return A[q]
    elif i < k:
        return randomized_select(A, p, q - 1, i)
    else:
        return randomized_select(A, q + 1, r, i - k)

In [124]:
def select(A, p, r, i):
    
    if p == r:
        return A[p]
    
    groups = group(A[p:r + 1], 5)    
    group_medians = []
    
    for a_group in groups:
        group_medians.append(simple_median(a_group))
        
    median_of_medians = select(np.array(group_medians), 0, len(group_medians) - 1, (len(group_medians) - 1) // 2)
    
    x_index = index_of(A, median_of_medians)
    
    q = modified_partition(A, p, r, x_index)
    k = q - p + 1
    
    if i == k:
        return A[q]
    elif i < k:
        return select(A, p, q - 1, i)
    else:
        return select(A, q + 1, r, i - k)    

In [ ]:
def randomized_weighted_select(A, W, p, r, i):
    
    if p == r:
        return A[p]
    
    q = randomized_partition(A, p, r)
    

In [152]:
A = [2, 1, 16, 18, 12, 11, 15, 8, 9, 20]

W = np.abs(np.random.randn(10))
tot = np.sum(W)
W = W / tot
W = np.around(W, 2)

In [153]:
def naive_weighted_median(A, W):
    
    inds = np.argsort(A)
    
    A = np.array(A)[inds]
    W = np.array(W)[inds]
    
    sm, i = 0, 0
    while sm < 1/2:
        
        sm += W[i]
        i += 1
    
    return A[i - 1]

In [154]:
naive_weighted_median(A, W)

12

In [155]:
print(A)
print(W)

[2, 1, 16, 18, 12, 11, 15, 8, 9, 20]
[0.31 0.07 0.28 0.05 0.07 0.02 0.06 0.05 0.01 0.08]


In [ ]:
def weighted_select(A, p, r, i)